In [ ]:
import os
import cv2
import numpy as np
import time
import import_ipynb
from IPython.display import clear_output
from grabscreen import grab_screen
from inputManager import get_input
from keras import Model
from keras.models import load_model
from vjoy import vJoy, ultimate_release

In [ ]:
# model loading
model = load_model('models/modelX.h5')

In [ ]:
tmpTime = time.time()
#######################################
# L,T,R,B
Window_Width = 1920
Window_Height = 835
grabBox = (0,40,Window_Width,Window_Height)
navi_size = (0,835,306,1080)

# Window_Width = 800
# Window_Height = 463
# grabBox = (0,40,Window_Width,Window_Height)
# navi_size = (0,463,128,600)

time_per_screen = 0.15
#######################################
XYRANGE = 16393
ZRANGE = 32786
vj = vJoy()
gtav_helper_log = open("C:\\Program files\Cheat Engine 7.1\log.txt","r")
objects_cascade = cv2.CascadeClassifier('haarcascade_car.xml')

def Set_Driving_Input(screen, speed):
    pred = model.predict(screen)
    clear_output(wait=True)
    print("Speed Nrmlzd:")
    print(speed)
    print("pr_spd     FB          LR")
    print(pred[0])
    pr_speed  = pred[0][0]
    forw_back = pred[0][1]
    left_right= pred[0][2]
    
    if speed>pr_speed:
        # slow down
        forw_back = forw_back - (speed-pr_speed)/80
        if forw_back<0: forw_back = 0
    else:
        # speed up
        forw_back = forw_back + (pr_speed-speed)/80
        if forw_back>1: forw_back = 1
    
    vj.open()
    if forw_back>0.5:
        joystickPosition = vj.generateJoystickPosition(
            # left and right trigger
            wAxisZ   =int(0),
            wAxisZRot=int(ZRANGE*(forw_back-0.5)*2),
            # Left thump x axis (left positive right negative)
            wAxisX   =int(XYRANGE + ((1-2*left_right))*XYRANGE)
        )
    else:
        joystickPosition = vj.generateJoystickPosition(
            wAxisZ   =int(ZRANGE*forw_back*2),
            wAxisZRot=int(0),
            wAxisX   =int(XYRANGE + ((1-2*left_right))*XYRANGE)
        )
    vj.update(joystickPosition)
    vj.close()

Manual_Control = False
ultimate_release()
while(True):
    cv2.imshow('TST',np.zeros((150,150)))
    if not Manual_Control:
         # Taking screen image
        screen = np.array(grab_screen(region=grabBox))
        screen = cv2.cvtColor(screen,cv2.COLOR_BGR2GRAY)
        #cv2.imshow('TST',screen)
        screen = cv2.resize(screen, (220, 96))
        #cv2.imshow('TST',screen)
        screen = screen.reshape(-1,96,220,1)
        # Taking speed
        gtav_helper_log.seek(0)
        speed = float(gtav_helper_log.read(5)) / 67.21
        
        Set_Driving_Input(screen, speed)
        if (time.time() - tmpTime < time_per_screen):
            time.sleep(time.time() - tmpTime)
        tmpTime=time.time()
        
    if 'T' in get_input():
        Manual_Control = not Manual_Control
        ultimate_release()
        time.sleep(0.5)
        
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        ultimate_release()
        break

In [ ]:
# release all inputs on virtual joystick
from vjoy import vJoy, ultimate_release

ultimate_release()